# Introduction to Skyway and Some Canvas Tools for Automating Training Data Creation from Canvas Data 

This notebook demonstrates some of the usages of the [skyway](https://github.com/rjpolackwich/skyway) library, both in terms of accessing/understanding/rasterizing data from canvas buckets as well as building sets of binary custom OSM-feature-labeled data by pointing the library to a canvas bucket. The skyway library can be installed via conda or pip, and requires python >= 3.7.x


There are some additional, optional installs that this notebook uses for visualization and mapping, which can be found immediately below. These dependencies will eventually be added as optional deps to the skyway package or somewhere else to support Jupyter Notebook/ Jupyter Lab development environments.

In [ ]:
# pip install maxar-skyway
# conda install -c digitalglobe skyway

In [1]:
%matplotlib inline

# Visualization dependencies
import matplotlib.pyplot as plt

from IPython.display import display
from sidecar import Sidecar
from ipyleaflet import (
    Map,
    basemaps,
    basemap_to_tiles,
    ImageOverlay,
    Polyline, Rectangle, Polygon,
    LayerGroup, GeoJSON, Marker,
    WidgetControl, ScaleControl, LayersControl
)
from ipywidgets import HTML

### First Steps: Point skyway to a Canvas Bucket

The easiest way to start working with and exploring canvas data is to simply import the `CanvasCollection` object from skyway, and point it to an AWS bucket-like path where canvas data is stored. This can be a remote (AWS s3) bucket, or a directory tree on a local filesystem.

In [2]:
from skyway.canvas import CanvasCollection

coll = CanvasCollection(bucket="/Users/jamiepolackwich1/dg/canvas-buckets/maxar-analytics-data/canvas_v2_sample", local=True)

The assigned variable `coll` represents a CanvasCollection instance. This is intended to provide accessors to the underlying data, as well as a general overview of the available data, where it is in the world, how much there is, etc. A few methods can be useful for understanding what you're working with.

In [3]:
coll.descriptions()

19
Total data area covered: 2150000000 m^2
Total number of quadkey zones: 86
Total number of catalog ids tiled: 12


42
Total data area covered: 1800000000 m^2
Total number of quadkey zones: 72
Total number of catalog ids tiled: 15




the `descriptions` method tells us what UTM zones the canvas bucket contains data for, how much coverage, and the catalog IDs that make up those data tiles. To access one of these zones, we can call `.zone` to get a more specific kind of data accessor.

In [4]:
tz42 = coll.zone(42)
tz19 = coll.zone(19)
tz = tz42
print(type(tz42))

<class 'skyway.canvas.CanvasZone'>


The above objects are useful for accessing data as well as statistics and information from the data bucket. These are instances of `skyway.CanvasZone`, and can be used to access the underlying tiling scheme used to generate the tiles in your canvas bucket. There are many useful methods on these items, and more detailed information about how they can be used can be found in the skyway documentation. A few are shown below:

A few `CanvasZone` methods:

In [5]:
print("Number of Geotiles making up zone: {}".format(tz.nqks))
print("Number of Catalog ids making up zone: {}".format(tz.ncatids))

Number of Geotiles making up zone: 72
Number of Catalog ids making up zone: 15


We can generate lists of the actual quadkeys and catalog ids that underly this UTM zone in this Canvas Bucket:

In [6]:
print(tz.tile_quadkeys)
print("\n")
print(tz.tile_cogs)
print("\n")

['120200000003', '120200000231', '120200000200', '120200000032', '120200000033', '120200000201', '120200000230', '120200000002', '120200000112', '120200000320', '120200000311', '120200000123', '031311111113', '031311111310', '120200000122', '120200000310', '120200000321', '120200000113', '031311111311', '031311111112', '031311111130', '120200000131', '120200000303', '120200002001', '031311111131', '120200000101', '120200002000', '120200000302', '120200000130', '120200000020', '120200000212', '120200000223', '120200000222', '120200000213', '120200000021', '120200000232', '120200000031', '120200000203', '120200000202', '120200000030', '120200002100', '120200000233', '031311111313', '120200002010', '120200000120', '120200000312', '031311111312', '120200000313', '120200000121', '120200002011', '120200000110', '120200000322', '120200000300', '120200000132', '120200000103', '120200000331', '031311111133', '031311111330', '120200000330', '120200000102', '120200000133', '120200000301', '031311

The `.tiler` object on the canvas zone has helpful methods for understanding things like physical lengths, image dimensions, and corresponding zoom parameters and other relative inputs. This API is subject to change.

In [7]:
print("Length in meters at tile zoom 15:  {}".format(tz.tiler.tile_length_at_zoom(15)))
print("Dimensions of Image at zoom level 15:  {}".format(tz.tiler.dims_from_zoom(15)))
print("\n")

Length in meters at tile zoom 15:  625.0
Dimensions of Image at zoom level 15:  2048.0




We can get tile objects as well:

In [8]:
tile = tz.tiler.tile_from_quadkey(tz.tile_quadkeys[0])
print(tile.quadkey)

120200000003


The tile object is effectively a wrapper around `tiletanic.base.Tile` and the tiletanic tilescheme classes, and provides methods for making Geographic transformations as well as accessing parent and child tiles and information, etc

In [9]:
%%time
#kids = containing_tiles(tz.tiler, tile, [17])
c = tile.children()

CPU times: user 1.39 ms, sys: 1.12 ms, total: 2.51 ms
Wall time: 1.48 ms


In [10]:
cc = c[0]

In [11]:
tile.zoom

12

## Second Steps: Build an OSM query

We can import the useful `skyway.QueryBuilder` object in order to easily build an Overpass Query Language query that we can then request over some spatial area. This can be useful to run over tiles at zoom size 12. The QueryBuilder section of the skyway package is in development and is intended to be useful for building the kinds of data insight and access patterns described elegantly in this [blog post](https://dev.overpass-api.de/blog/index.html), which provides an overview of some of the more complicated ways that the newest deployed version of the [Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API) and is authored by a core Overpass maintainer.

In [12]:
from skyway.query import make_opql_query

qs = make_opql_query([5, 10, 15, 20],
                    elements=["node", "way", "relation"],
                    osm_tags="railway",
                    tag_vals=["rail"],
                    fmt="json",
                    timeout=25,
                    dry_run=True)

print(qs)

[out:json][timeout:25];(node["railway"="rail"][5,10,15,20];);out;>;out skel qt;


More complicated query objects can be built using the `QueryBuilder` object; for instance, you can ask for multiple tags with multiple values at the same time:


In [33]:
# from skyway.query import QueryBuilder
qb = QueryBuilder()
q = qb.from_query(osm_tags=["railway"])
q.add_matching_values(["rail", "route", "lightrail", "monorail", "tram"])
qb.add_query(q)
qb.add_aoi([...])

We can utilize some of the patterns in the above mentioned blog to, for instance, learn about the differences in OSM feature distribution across various AOIs (todo)

In [37]:
#from skyway.query import StatQuery
sq = StatQuery()
q = sq.from_query(osm_tags["railway"])
sq.add_query(q)
sq.add_aoi([...])

df = sq.get_top_100().to_DataFrame()

The above code can tell us what the top 100 tags associated with "railway" are in a particular AOI. There are many other options for building useful queries, and they are explored in more depth in the skyway documentation.

## Third Step - Combine: Run a query across a Canvas AOI

Because of the "Geo" nature of the `Tile` objects and the iterative nature of the CanvasCollection objects and their child classes, we can build a relevant OSM query object right from some input parameters; combined with a specified Canvas AOI and zoom level, (or quadkeys, etc),  we can run this query over our dataspace.

In [39]:
# from skyway.transformers import WGS84Transformer
for tile in tz:
    with WGS84Transformer(tile) as t:
        print(t.bounds)
        
print("\n\n\n")

(69.27283246333374, 34.65649964003097, 69.32757573811152, 34.70172302264596)
(69.3273982338682, 34.656340917523536, 69.38217069663621, 34.70158849318344)
(68.89086655512233, 34.61180050624475, 68.94546276396125, 34.65692697659191)
(68.9454332447363, 34.611837074247234, 69.00000000000003, 34.65693918635222)
(69.00000000000003, 34.611837074247234, 69.05456675526376, 34.65693918635222)
(69.05453723603883, 34.61180050624475, 69.10913344487774, 34.65692697659191)
(69.1090744065704, 34.61173955973388, 69.16370000319267, 34.65689034735906)
(69.16361144608796, 34.61165423495451, 69.2182663645603, 34.65682929879799)
(69.21814828908576, 34.611544532242505, 69.27283246333374, 34.656743831149136)
(69.27268487005948, 34.61141045202966, 69.3273982338682, 34.65663394474918)
(69.32722112350679, 34.611251994843734, 69.38196361052127, 34.65649964003097)
(69.38175698392776, 34.611069161308436, 69.43652852765351, 34.656340917523536)
(68.89092559342967, 34.566710331172594, 68.94549221767883, 34.61183707424

We can set our rail query using a python partial, then pass in our aoi's as kwargs to the resulting funtion - see below:

In [40]:
rail_q = partial(make_opql_query, osm_tags="railway", tag_vals="rail", timeout=25, fmt="json")

In [36]:
for tile in tz:
    with WGS84Transformer(tile) as t:
        # r = rail_q(t.bounds)
        
        

(520000.0, 3795000.0, 525000.0, 3800000.0)

### The following code is in the creative stages

In [ ]:
# %%time
# polys = []
# fc = {"type": "FeatureCollection",
#       "features": list()}
# for gtile in tz42:
#     with WGS84Transformer(gtile) as t:
#         fc["features"].append(to_gjson(t))
#         polys.append(t.asShape)


#pgons = geom.MultiPolygon(polys)
#data = geom.mapping(pgons)

# def make_ovp_query(aoicoords, tag, values=[]):
#     q = overpass.ql_query(aoicoords, tag=tag, values=values)
#     print(q)
#     r = overpass.request(q)
#     return r

# query = overpass.ql_query((59.648316096391, 29.64111328125, 60.228902767513, 31.09130859375), tag='railway', values=['rail'])
# response = overpass.request(query)



# _bounds = reverse_coordorder(tile.bounds)
# q = overpass.ql_query(_bounds, tag='highway', values=['unclassified'])# 'primary,secondary,tertiary'])
# #q = overpass.ql_query(_bounds, tag='building', values=['yes'])
# r = overpass.request(q)

# gdf = gpd.GeoDataFrame.from_features(overpass.as_geojson(r, 'linestring'))
# gdf.plot()

In [ ]:
# path = tz42.qk_path + "/{}".format(qk)
# print(path)
# tifs = [f for f in os.listdir(path) if f[-4:] == '.tif']
# print(tifs)

# catid = "1040010058D6EF00"
# impath = path + "/" + catid + "-visual.tif"
# print(impath)


# #f, axs = plt.subplots(4, 4, figsize=(16,16), subplot_kw={'xticks': [], 'yticks': []})
# with rasterio.open(impath) as src:
#     plt.figure(figsize=(10, 10))
#     arr = src.read(out_shape=(3, int(src.height / 16), int(src.width / 16)))[..., 32:1056, 32:1056]
#     rashow(arr)


In [ ]:
# try:
#     del m
# except:
#     pass

# center = (tz42.centroid.y, tz42.centroid.x)
# m = Map(center=center, zoom=8)
# geo_json = GeoJSON(data=fc, style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1})
# # marker = Marker(location=center, draggable=False)
# # m.add_layer(marker);
# m.add_layer(geo_json);

# html = HTML('''Hover over a tile''')
# html.layout.margin = '0px 20px 20px 20px'
# control = WidgetControl(widget=html, position='topright')
# m.add_control(control)

# def update_html(feature, **kwargs):
#     html.value = '''
#     <h3><b>{}</b><h3>
#     <h4>Tile Index: ({}, {})</h4>
#     '''.format(feature['properties']['tile_id'],
#                feature['properties']['ix'],
#                feature['properties']['iy'])
    
# geo_json.on_hover(update_html)

# sc = Sidecar(title="whatever")
# with sc:
#     display(m)

In [ ]:
# /*
# This has been generated by the overpass-turbo wizard.
# The original search was:
# “historic=* or leisure=*”
# */
# [out:json][timeout:25];
# // gather results
# (
#   // query part for: “historic=*”
#   node["historic"]({{bbox}});
#   way["historic"]({{bbox}});
#   relation["historic"]({{bbox}});
#   // query part for: “leisure=*”
#   node["leisure"]({{bbox}});
#   way["leisure"]({{bbox}});
#   relation["leisure"]({{bbox}});
# );
# // print results
# out body;
# >;
# out skel qt;

# /*
# This has been generated by the overpass-turbo wizard.
# The original search was:
# “historic=* or leisure=*”
# */
# [out:json][timeout:25];
# // gather results
# (
#   // query part for: “historic=*”
#   node["historic"](-34.480382504358694,-58.64295959472656,-34.38467103941415,-58.518333435058594);
#   way["historic"](-34.480382504358694,-58.64295959472656,-34.38467103941415,-58.518333435058594);
#   relation["historic"](-34.480382504358694,-58.64295959472656,-34.38467103941415,-58.518333435058594);
#   // query part for: “leisure=*”
#   node["leisure"](-34.480382504358694,-58.64295959472656,-34.38467103941415,-58.518333435058594);
#   way["leisure"](-34.480382504358694,-58.64295959472656,-34.38467103941415,-58.518333435058594);
#   relation["leisure"](-34.480382504358694,-58.64295959472656,-34.38467103941415,-58.518333435058594);
# );
# // print results
# out body;
# >;
# out skel qt;

#[out:json][timeout:25];(node["historic"]({south},{west},{north},{east});way["historic"]({south},{west},{north},{east});relation["historic"]({south},{west},{north},{east});node["leisure"]({south},{west},{north},{east});way["leisure"]({south},{west},{north},{east});relation["leisure"]({south},{west},{north},{east}););out body;>;out skel qt;
 
    
# {{OverpassTurboExample|loc=-34.43247;-58.58065;13|query=
# /*
# This has been generated by the overpass-turbo wizard.
# The original search was:
# “historic=* or leisure=*”
# */
# [out:json][timeout:25];
# // gather results
# (
#   // query part for: “historic=*”
#   node["historic"]({{((}}bbox{{))}});
#   way["historic"]({{((}}bbox{{))}});
#   relation["historic"]({{((}}bbox{{))}});
#   // query part for: “leisure=*”
#   node["leisure"]({{((}}bbox{{))}});
#   way["leisure"]({{((}}bbox{{))}});
#   relation["leisure"]({{((}}bbox{{))}});
# );
# // print results
# out body;
# >;
# out skel qt;
# }}



In [ ]:
# %matplotlib inline

# import os
# import sys
# import typing
# import json
# import math
# import datetime
# from functools import partial, partialmethod
# import typing_extensions as typing
# #import typing
# import types
# import dataclasses
# from pathlib import Path
# import matplotlib.pyplot as plt

# from IPython.display import display
# from sidecar import Sidecar
# from ipyleaflet import (
#     Map,
#     basemaps,
#     basemap_to_tiles,
#     ImageOverlay,
#     Polyline, Rectangle, Polygon,
#     LayerGroup, GeoJSON, Marker,
#     WidgetControl, ScaleControl, LayersControl
# )
# from ipywidgets import HTML


# import boto3
# import botocore
# import s3fs

# import numpy as np
# import networkx as nx
# import rasterio
# from rasterio.plot import show as rashow
# import shapely.ops as ops
# import shapely.geometry as geom
# from pyproj import CRS, Proj, Transformer

# import tiletanic as tt
# import canvas

# from osmxtract import overpass, location
# import geopandas as gpd

# import requests

In [ ]:
# orm = {
#     "url": 'http://{s}.tiles.openrailwaymap.org/standard/{z}/{x}/{y}.png',
#     "min_zoom": 2,
#     "max_zoom": 19,
#     "attribution": '<a href="https://www.openstreetmap.org/copyright">© OpenStreetMap contributors</a>, Style: <a href="http://creativecommons.org/licenses/by-sa/2.0/">CC-BY-SA 2.0</a> <a href="http://www.openrailwaymap.org/">OpenRailwayMap</a> and OpenStreetMap',
#     "name": "OpenStreetMap.OpenRailwayMap"
# }

# _basemaps = basemaps.copy()
# basemaps.OpenStreetMap['OpenRailwayMap'] = orm

In [5]:
# Bounds are always (lonmin, latmin, lonmax, latmax) from shit
# eg (xmin, ymin, xmax, ymax)
# WORLD_CRS = CRS.from_epsg(4326)
# MAP_CRS = CRS.from_epsg(3857)

# class GeoTileTransformer(object):
#     def __init__(self, tile, to_crs):
#         self.tile = tile
#         self.to_crs = to_crs
        
#     def __enter__(self):
#         self.tile.set_crs(self.to_crs)
#         return self.tile
    
#     def __exit__(self, *args):
#         self.tile.reset_crs()
        
        
        
# class WGS84Transformer(GeoTileTransformer):
#     def __init__(self, tile):
#         GeoTileTransformer.__init__(self, tile, WORLD_CRS)

        

# class GeoTile(object):
#     dtype = [('ix', int), ('iy', int), ('zoom', int)]
    
#     def __init__(self, xi, yi, zoom, tiler=None):
#         self.xi = xi
#         self.yi = yi
#         self.zoom = zoom
#         self._tiler = tiler
#         self._qk = None
#         if self.quadkey[0] in ('0', '1'):
#             self.__crs__ = tiler._crs_n
#             self._tr = tiler._tr_n
#         else:
#             self.__crs__ = tiler._crs_s
#             self._tr = tiler._tr_s
#         self._crs = self.__crs__
#         self.__gi__ = geom.box(*self.utm_bounds).__geo_interface__
#         self._gi = self.__gi__
        
#     @property
#     def utm_zone(self):
#         return self._tiler.utm_zone
        
#     def parent(self):
#         return self._tiler.tile_parent(self.asTile)
    
#     def children(self):
#         return self._tiler.tile_children(self.asTile)
    
#     @property
#     def quadkey(self):
#         if self._qk is None:
#             self._qk = self._tiler.quadkey_from_tile(self.asTile)
#         return self._qk
    
#     @property
#     def asTile(self):
#         return tt.base.Tile(self.xi, self.yi, self.zoom)
    
#     @property
#     def crs(self):
#         return self._crs
    
#     def set_crs(self, to_crs):
#         ccrs = self._crs
#         self._crs = to_crs
#         if ccrs == to_crs:
#             return
#         if ccrs == self.__crs__ and to_crs == WORLD_CRS:
#             tr = self._tr
#         else:
#             tr = Transform.from_crs(ccrs, to_crs, always_xy=True).transform
#         self._gi = ops.transform(tr, geom.shape(self)).__geo_interface__
        
#     toWGS84 = partialmethod(set_crs, WORLD_CRS)
    
#     def reset_crs(self):
#         self._crs = self.__crs__
#         self._gi = self.__gi__
    
#     @property
#     def width_meters(self):
#         return self._tiler.tile_size_at_zoom(self.zoom)
    
#     @property
#     def length_meters(self):
#         return self.width_meters
    
#     @property
#     def utm_bounds(self):
#         return self._tiler._tiler.bbox(self.asTile)
    
#     @property
#     def bounds(self):
#         return geom.shape(self).bounds
    
#     @property
#     def __geo_interface__(self):
#         return self._gi
    
#     @classmethod
#     def _from_tile(cls, tile, tiler=None):
#         return cls(tile.x, tile.y, tile.z, tiler=tiler)
    
#     @property
#     def asShape(self):
#         return geom.asShape(self)
    
#     @property
#     def z(self):
#         return self.zoom
    
#     def __iter__(self):
#         return iter([self.xi, self.yi, self.zoom])
    
    



# class ProjectedUTMTiling(object):
            
#     def __init__(self, zone=None, tiler=None):
#         self.utm_zone = zone
#         self._tiler = tiler
#         self._tiling_zoom_level = tiler.zoom
#         self._tiling_physical_length = tiler.tile_size
#         self._tf = partial(GeoTile._from_tile, tiler=self)
#         self._crs_n = CRS.from_epsg(f'326{self.utm_zone:02d}')
#         self._crs_s = CRS.from_epsg(f'327{self.utm_zone:02d}')
#         self._tr_n = Transformer.from_crs(self._crs_n, WORLD_CRS, always_xy=True).transform
#         self._tr_s = Transformer.from_crs(self._crs_s, WORLD_CRS, always_xy=True).transform
        
#     def tile_from_xy(self, xcoord, ycoord, zoom=12):
#         tile = self._tiler.tile(xcoord, ycoord, zoom)
#         return self._tf(tile)
    
#     def tile_from_quadkey(self, qk):
#         tile = self._tiler.quadkey_to_tile(qk)
#         return self._tf(tile)
    
#     def quadkey_from_tile(self, *tile):
#         return self._tiler.quadkey(*tile)
    
#     def tile_parent(self, *tile):
#         tile = self._tiler.parent(*tile)
#         return self._tf(tile)
    
#     def tile_children(self, *tile):
#         children = self._tiler.children(*tile)
#         return [self._tf(tile) for tile in children]
    
#     def tile_length_at_zoom(self, zoom_level):
#         return self._tiling_physical_length / 2**(zoom_level - self._tiling_zoom_level)
    
#     def zoom_from_tile_dim(self, dimension):
#         pass

#     def __getattr__(self, atr):
#         return getattr(self._tiler, atr)
    
#     def dims_from_zoom(self, zoom, init_dim=16384, init_zoom=12):
#         return init_dim / 2**(zoom - init_zoom)
    

    
# # Accessor api

        
# class CanvasClient(object):
#     def __init__(self, bucket, local=True, s3conn=None):
#         self.bucket = bucket
#         self.local = local
#         if not local:
#             if s3conn is None:
#                 s3conn = s3fs.S3FileSystem(anon=False)
#         self.s3conn = s3conn
            
#     def list_dir(self, *args, **kwargs):
#         if self.local:
#             return os.listdir(*args, **kwargs)
#         return self.s3conn.ls(*args, **kwargs)
    

    
    
# def itrreduce(fn, iterable, initializer=None):
#     it = iter(iterable)
#     if initializer is None:
#         value = next(it)
#     else:
#         value = initializer
#     for element in it:
#         value = fn([value, element])
#     return value


# def to_gjson(tile):
#     geodata = geom.mapping(t)
#     gj = {"type": "Feature",
#          "properties": {"tile_id": "+".join([str(t.utm_zone), t.quadkey]),
#                         "ix": t.xi,
#                         "iy": t.yi,
#                         "zoom": t.zoom},
#          "geometry": geodata}
#     return gj

    
# class CanvasZone(CanvasClient):
    
#     def __init__(self, utm_zone, tiler=None, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.utm_zone = utm_zone
#         self.tiler = ProjectedUTMTiling(zone=utm_zone, tiler=tiler)
#         self.cvg = nx.Graph(store=self.bucket, utm_zone=utm_zone)
#         self.qk_path = os.path.join(self.bucket, str(utm_zone))
#         self._tile_quadkeys_fetched = False
#         self._tile_cogs_fetched = False
#         self._qksample = self.tile_quadkeys[0]
#         self._gtm = None
#         self.__gi__ = None
        
#     @property
#     def tile_quadkeys(self):
#         if not self._tile_quadkeys_fetched:
#             qk_paths = self.list_dir(self.qk_path)
#             for qkp in qk_paths:
#                 qk = Path(qkp).parts[-1]
#                 self.cvg.add_node(qk, obj="quadkey")
#             self._tile_quadkeys_fetched = True
#         return [n for n in self.cvg.nodes() if self.cvg.nodes[n]['obj'] == "quadkey"]
        
#     @property
#     def tile_cogs(self):
#         if not self._tile_cogs_fetched:
#             for qk in self.tile_quadkeys:
#                 cog_path = os.path.join(self.qk_path, qk)
#                 cog_files = self.list_dir(cog_path)
#                 for fp in cog_files:
#                     catid = Path(fp).stem.split("-")[0]
#                     self.cvg.add_node(catid, obj="catalog_id")
#                     self.cvg.add_edge(catid, qk)
#             self._tile_cogs_fetched = True
#         return [n for n in self.cvg.nodes() if self.cvg.nodes[n]['obj'] == 'catalog_id']
    
#     @property
#     def nqks(self):
#         return len(self.tile_quadkeys)
    
#     @property
#     def ncatids(self):
#         return len(self.tile_cogs)
    
#     @property
#     def area_coverage(self, units='m'):
#         return str((5_000 * 5_000) * self.nqks) + ' ' + units + '^2'
    
#     def __getitem__(self, quadkey):
#         if quadkey not in self.tile_quadkeys:
#             raise KeyError
#         return self.tiler.tile_from_quadkey(quadkey)
    
#     def __setitem__(self, item):
#         raise NotImeplementedError
        
#     def __iter__(self):
#         tiles = np.array([self.tiler._tiler.quadkey_to_tile(qk) 
#                           for qk in self.tile_quadkeys], dtype=GeoTile.dtype)
#         for tile in np.sort(tiles, order=['iy', 'ix']):
#             yield self.tiler._tf(tt.base.Tile(*tile))
            
#     def iter_geoms(self):
#         for tile in iter(self):
#             with WGS84Transformer(tile) as geotile:
#                 yield geotile.asShape
            
#     @property
#     def __geo_interface__(self):
#         if self.__gi__ is None:
#             self.__gi__ = itrreduce(ops.unary_union, 
#                                     self.iter_geoms()).__geo_interface__
#         return self.__gi__
    
#     @property
#     def asShape(self):
#         return geom.asShape(self)
    
#     @property
#     def centroid(self):
#         return self.asShape.centroid
        
    

# class CanvasCollection(CanvasClient):
#     def __init__(self, aoi=None, tilescheme=tt.tileschemes.WNUTM5kmTiling, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.aoi = aoi
#         self.tiler = tilescheme()
#         self._zlut = dict()
#         self._zone_paths_fetched = False
        
#     def canvas_zones(self):
#         if not self._zone_paths_fetched:
#             self._zone_paths = self.list_dir(self.bucket)
#             self._zone_paths_fetched = True
#             self.zones = sorted([int(Path(zp).parts[-1]) for zp in self._zone_paths if Path(zp).parts[-1] not in ('models', 'output')])
#         return self.zones
    
#     def zone(self, zone, *args, **kwargs):
#         if zone not in self.canvas_zones():
#             raise OSError("Zone {} not in path".format(zone))
#         if zone in self._zlut: return self._zlut[zone]
#         self._zlut[zone] = CanvasZone(zone, bucket=self.bucket, local=self.local, s3conn=self.s3conn, tiler=self.tiler, **kwargs)
#         return self._zlut[zone]  
    
#     def descriptions(self):
#         for z in self.canvas_zones():
#             print(z)
#             print("Total data area covered: " + self.zone(z).area_coverage)
#             print("Total number of quadkey zones: " + str(self.zone(z).nqks))
#             print("Total number of catalog ids tiled: " + str(self.zone(z).ncatids))
#             print("\n")
        
        
    

    
    

In [18]:
containing_tiles = tt.tilecover._containing_tiles
cover_polygonal = tt.tilecover._cover_polygonal
cover_geometry = tt.tilecover.cover_geometry

In [26]:
def make_opql_query(aoi,
                    elements=["node", "way", "relation"],
                    osm_tags="railway",
                    tag_vals="rail",
                    fmt="json",
                    timeout=25,
                    overpass_url="http://overpass-api.de/api/interpreter",
                    dry_run=False,
                    ):
    
    qsmeta = f'[out:{fmt}][timeout:{timeout}];'
    if isinstance(osm_tags, str):
        osm_tags = [osm_tags]*len(elements)
    if isinstance(tag_vals, str):
        tag_vals = [tag_vals]*len(elements)
    qsbody = '('
    for el, tag, val in zip(elements, osm_tags, tag_vals):
        qsbody += f'{el}["{tag}"="{val}"]{aoi};'.replace(' ', '')
    qsbody += ');'
    qsout = 'out;>;out skel qt;'
    qs = qsmeta + qsbody + qsout   
    if dry_run:
        return qs
    
    resp = requests.get(overpass_url, params={'data': qs})
    resp.raise_for_status()
    return resp.json()

In [1]:
a = ["hey", "hi", "hello"]

In [2]:
a.loc("hey")

AttributeError: 'list' object has no attribute 'loc'

In [4]:
a.index("hi")

1